In [ ]:
import numpy as np
import pandas as pd
import copy
import time

import gc
import ast

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_one_itemsets(transaction_db, min_sup):
    
    items = {}
    
    for t_tup in transaction_db.list_items:
        
        # Remove duplicates in list.
        # This is required because a transaction might have multiple occurences of an item.
        # We are not using occurence information in the vanilla algorithm.
        new_t = []
        [new_t.append(x) for x in t_tup[0] if x not in new_t]
        
        for i in new_t:
            if i in items.keys():
                items[i] += t_tup[1]
            else:
                items[i] = t_tup[1]
    
    # Eliminate one itemsets which do not meet minimum support criteria
    items = {i:sup for (i,sup) in items.items() if sup >= min_sup}
    
    return items

In [ ]:
def sort_transaction(t, one_itemset_dict):
    sorted_t = sorted(one_itemset_dict.keys(), key=one_itemset_dict.get, reverse=True)
    return [i for i in sorted_t if i in t]

In [ ]:
def tree_contains_node(name, parent, fp_tree):
    for node in fp_tree:
        if (node['name'] == name) and (node['parent'] is parent):
            return node
    
    return False

In [ ]:
def insert_tree(prefix, suffix, current_root, fp_tree, node_link_dict, transaction_count):
    
    # Check if route to the prefix node exists in tree
    # If exists, get the node
    new_root = tree_contains_node(prefix, current_root, fp_tree)
    
    if new_root == False:
        # If route doesn't exist, create new child node to current root
        new_root = {'name':prefix, 'count':transaction_count, 'parent': current_root}
        
        # Add the link to node link dictionary
        node_link_dict[prefix].append(new_root)
        
        # Add new node to tree
        fp_tree.append(new_root)
    else:
        # If route exists, increase its count by 1
        new_root['count'] += transaction_count
    
    # If reached the end of transaction, stop
    if len(suffix) == 0:
        return
    
    # Recursively call itself
    insert_tree(suffix[0], suffix[1:], new_root, fp_tree, node_link_dict, transaction_count)

In [ ]:
def fp_growth(transaction_db, min_sup, fp_list, prefix = [], output=True):
    
    ####### PART A: GENERATE FREQUENT PATTERNS AND SORT TRANSACTIONS #######
    
    # Get one itemsets which meet minimum support
    one_itemset_dict = get_one_itemsets(transaction_db, min_sup)
    
    # Concanenate frequent patterns with prefix
    freq_patterns = []
    for key,val in one_itemset_dict.items():
        pattern = prefix + [key]
        freq_patterns.append((pattern,val))
    
    if output:
        fp_list.append(freq_patterns)
    else:
        for pat in freq_patterns:
            if pat not in fp_list:
                fp_list.append(pat[0])
    
    if output:
        print("\n\nFrequent patterns generated: ", freq_patterns)
    
    # Sort items within transactions in descending order and eliminate

    if output:
        print(f"\n\nCurrent Pattern base for prefix {prefix}: ", end='')
        display(transaction_db)

    # Initialize node link dictionary
    # This will store node links to each item in tree
    node_link_dict = {item:[] for item in one_itemset_dict.keys()}

    # Create tree data structure as a list of dictionary nodes
    # Each node is {name: item-name, count: count, parent: parent-node}
    # Insert root node as {name:None, count:None, parent: None}
    # The traversal count will be used in Part C (mining the tree)
    fp_tree = [{'name':"Root", 'count':None, 'parent': None}]
    
    
    ####### PART B: GENERATE FP TREE #######
    
    # Generate FP tree by scanning over all sorted transactions
    root_node = fp_tree[0]
    for t_tup in transaction_db.list_items:
        
        # Sort t according to 1-itemset support counts
        # This is required only at the root node
        if prefix == []:
            t = sort_transaction(t_tup[0], one_itemset_dict)
        else:
            t = [i for i in t_tup[0] if i in one_itemset_dict.keys()]
        
        if len(t) > 1:
            insert_tree(t[0], t[1:], root_node, fp_tree, node_link_dict, t_tup[1])

    if output:
        print(f"\n\nGenerated FP-Tree for prefix {prefix}: \n")
        print_fp_tree(fp_tree)
    
    
    ####### PART C: GENERATE CONDITIONAL PATTERN BASE #######
    
    for item in one_itemset_dict.keys():
        
        cpb_transactions = []
        cond_pattern_base = []
        
        # Iterate over all node links of item
        for node in node_link_dict[item]:

            cur_node = node['parent']
            cur_route = []

            if cur_node is root_node:
                continue

            while cur_node is not root_node:
                cur_route.insert(0, cur_node)
                cur_node = cur_node['parent']

            cond_pattern_base.append((cur_route, node['count']))
            
            cpb_transactions.append(([path['name'] for path in cur_route], node['count']))
        
        if len(cond_pattern_base) > 0:
            if output:
                print(f"\n\nCPB for {item}: ", end='')
                print_cpb(cond_pattern_base)
        
        
        ####### PART D: RECURSE ON CONDITIONAL PATTERN BASE #######
        
        cpb_db = pd.DataFrame({'list_items':cpb_transactions})
        
        if cpb_db.shape[0] > 0:
            fp_growth(cpb_db, min_sup, fp_list, prefix + [item], output)

In [ ]:
data=pd.read_csv('withtexas.csv')

In [ ]:
data.isna().sum()

In [ ]:
transactions=[]
for _,row in data.iterrows():
    t = []
    t.append(row['gender'])
    t.append(row['age'])
    #t.append(row['org'])
    if not str(row['race'])=='nan':
        t.append(row['race'])
    if row['texas']==1:
        t.append('texas')
    transactions.append(t)

In [ ]:
tdf=pd.DataFrame({'list_items':transactions})
tdf['list_items'] = tdf['list_items'].apply(lambda x: (x,1))
tdf.head()

In [ ]:
# List to store frequent patterns
frequent_patterns = []

# Minimum support threshold
min_sup = 0.08 * tdf.shape[0]

start_t = time.perf_counter()

# Run FP growth
fp_growth(tdf, min_sup, frequent_patterns, output=False)

end_t = time.perf_counter()

print(f"Running time of FP-Growth = {end_t - start_t} seconds")

print(f"Number of frequent patterns generated by FP-Growth = {len(frequent_patterns)}\n")
frequent_patterns